In [1]:
import xlrd
import os
import random
import jieba
import pickle
from collections import Counter

In [2]:
workbook = xlrd.open_workbook(r'./data/taxdata3_3.xlsx')

# 获取第1张表
sheet1 = workbook.sheet_by_index(0)
sheet1_cols0 = sheet1.col_values(0) 
sheet1_cols1 = sheet1.col_values(1) 
sheet1_cols2 = sheet1.col_values(2) 
sheet1_cols3 = sheet1.col_values(3) 
sheet1_class_list = sheet1.col_values(4)
sheet1_class_list = sheet1_class_list[1:]
sheet1_col_length = len(sheet1_cols0)
raw_data1_list = []
for i in range(1, sheet1_col_length):
    k = sheet1_cols0[i] + sheet1_cols1[i] + sheet1_cols2[i] + sheet1_cols3[i]
    raw_data1_list.append(k.replace("\n",""))

# 获取第2张表
sheet2 = workbook.sheet_by_index(1)
sheet2_cols0 = sheet2.col_values(0) 
sheet2_cols1 = sheet2.col_values(1) 
sheet2_cols2 = sheet2.col_values(2) 
sheet2_cols3 = sheet2.col_values(3) 
sheet2_class_list = sheet2.col_values(4)
sheet2_class_list = sheet2_class_list[1:]
sheet2_col_length = len(sheet2_cols0)
raw_data2_list = []
for i in range(1, sheet2_col_length):
    k = sheet2_cols0[i] + sheet2_cols1[i] + sheet2_cols2[i] + sheet2_cols3[i]
    raw_data2_list.append(k.replace("\n",""))

# 获取第3张表
sheet3 = workbook.sheet_by_index(2)
sheet3_cols0 = sheet3.col_values(0) 
sheet3_cols1 = sheet3.col_values(1) 
sheet3_cols2 = sheet3.col_values(2) 
sheet3_cols3 = sheet3.col_values(3) 
sheet3_class_list = sheet3.col_values(4)
sheet3_class_list = sheet3_class_list[1:]
sheet3_col_length = len(sheet3_cols0)
raw_data3_list = []
for i in range(1, sheet3_col_length):
    k = sheet3_cols0[i] + sheet3_cols1[i] + sheet3_cols2[i] + sheet3_cols3[i]
    raw_data3_list.append(k.replace("\n",""))

#data 集合
raw_data_list = []
raw_data_list.extend(raw_data1_list)
raw_data_list.extend(raw_data2_list)
raw_data_list.extend(raw_data3_list)

#class 集合
class_list = []
class_list.extend(sheet1_class_list)
class_list.extend(sheet2_class_list)
class_list.extend(sheet3_class_list)

In [3]:
'''
综合税种=综合，两个类的、所得税=综合，发票管理、发票管理管理=发票，财务类=企业会计准则，财务类，
个税=个人所得税，环境保护税=环保税，土地使用税=城镇土地使用税，
教育费附加、城建税、教育费附加=附加税费，
出口退税=进出口税收，税收风险=税务风险
'''
for i in range(len(class_list)):
    if class_list[i] == '' or class_list[i] == '工业企业':
        class_list[i] = "未分类"
    if class_list[i]=='综合税种' or class_list[i]=='多税种' or class_list[i]=='所得税' or (' ' in class_list[i]) or ('，' in class_list[i]) or ('、' in class_list[i]) or ('\u3000' in class_list[i]):
        class_list[i] = "综合"
    class_list[i] = class_list[i].replace(' ', '')
    class_list[i] = class_list[i].replace('\n', '')
    if class_list[i] == '发票管理' or class_list[i] == '发票管理管理':
        class_list[i] = "发票"
    if class_list[i] == '企业会计准则，财务类':
        class_list[i] = "财务类"
    if class_list[i] == '土地使用税':
        class_list[i] = "城镇土地使用税"
    if class_list[i] == '附加税' or class_list[i] == '教育费附加' or class_list[i] == '城建税':
        class_list[i] = "附加税费"
    if class_list[i] == '出口退税':
        class_list[i] = "进出口税收"   
    if class_list[i] == '税收风险':
        class_list[i] = "税务风险"
    if class_list[i] == '印花':
        class_list[i] = "印花税"
    if class_list[i] == '纳税社保' or class_list[i] == '社保' or class_list[i] == '个税' or class_list[i] == '个人所得说' or class_list[i] == '新个税':
        class_list[i] = "个人所得税"   
    if class_list[i] == '营改增' or class_list[i] == '增值况':
        class_list[i] = "增值税"    
    if class_list[i] == '环保税':
        class_list[i] = "环境保护税" 
    if class_list[i] == '企业所得呼'or class_list[i] == '企业所得税法':
        class_list[i] = "企业所得税" 
    if class_list[i] == '土增税':
        class_list[i] = "土地增值税"
    if class_list[i] == '营改增':
        class_list[i] = "增值税"
    if class_list[i] == '税收优惠' or class_list[i] == '税务登记' or class_list[i] == '税务风险' or class_list[i] == '税务变更' or class_list[i] == '税务注销' or class_list[i] == '税务征管':
        class_list[i] = "税收征管"
    if class_list[i] == '申报征收':
        class_list[i] = '纳税申报'
    if class_list[i] == '关税':
        class_list[i] = "涉外税收"
        
c = Counter(class_list)
topn_class = 4       #取最多class的类别
data_number = 1001     #取data数目
class_type_list = [k[0] for k in c.most_common(topn_class)]
class_type_len = len(list(c.most_common(topn_class)))
print("class type total:", class_type_len)

data_class_list = list(zip(raw_data_list, class_list))  # zip压缩合并，将数据与标签对应压缩
random.shuffle(data_class_list)  # 将data_class_list乱序 
raw_data_list, class_list = zip(*data_class_list)  


#取最多类别的前n个样本
new_class_list = []
new_data_list = []
#todo: 每个类别取k个样本试试看
count_n = [1 for i in range(class_type_len)]        
for i in range(class_type_len):
    for j in range(len(class_list)):
        if class_list[j] == class_type_list[i] and count_n[i] < data_number:
            new_class_list.append(class_list[j])
            new_data_list.append(raw_data_list[j])
            count_n[i] += 1

class type total: 4


In [4]:
c.most_common(14)

[('企业所得税', 2502),
 ('增值税', 2428),
 ('个人所得税', 1611),
 ('综合', 1055),
 ('发票', 854),
 ('税收征管', 759),
 ('财务类', 621),
 ('印花税', 418),
 ('房产税', 309),
 ('进出口税收', 294),
 ('消费税', 239),
 ('契税', 214),
 ('未分类', 213),
 ('土地增值税', 210)]

In [5]:
print(new_class_list[199:201])
print(new_data_list[199:201])

['企业所得税', '企业所得税']
['企业发生的哪些成本可以在企业所得税税前扣除？    根据《中华人民共和国企业所得税法实施条例》(中华人民共和国国务院令2007年第512号)第二十九条规定：“企业所得税法第八条所称成本，是指企业在生产经营活动中发生的销售成本、销货成本、业务支出以及其他耗费。”   ——国务院令2007年第512号《中华人民共和国企业所得税法实施条例》第二十九条', '对中国保险保障基金有限责任公司哪些收入免征企业所得税？    根据《财政部 税务总局关于保险保障基金有关税收政策问题的通知》(财税〔2018〕41号)规定：“一、对中国保险保障基金有限责任公司(以下简称保险保障基金公司)根据《保险保障基金管理办法》(以下简称《管理办法》)取得的下列收入，免征企业所得税：\u3000\u30001.境内保险公司依法缴纳的保险保障基金；\u3000\u30002.依法从撤销或破产保险公司清算财产中获得的受偿收入和向有关责任方追偿所得，以及依法从保险公司风险处置中获得的财产转让所得；\u3000\u30003.接受捐赠收入；\u3000\u30004.银行存款利息收入；\u3000\u30005.购买政府债券、中央银行、中央企业和中央级金融机构发行债券的利息收入；\u3000\u30006.国务院批准的其他资金运用取得的收入。”——财税〔2018〕41号《财政部 税务总局关于保险保障基金有关税收政策问题的通知》']


In [6]:
#分词
new_data_list_cut = []
rowlength = len(new_data_list)
for i in range(rowlength):
    word_cut = jieba.cut(new_data_list[i], cut_all=False)
    word_list = list(word_cut) 
    new_data_list_cut.append(word_list)


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/wn/7md2z8rj1td41mc56_6lr3jr0000gn/T/jieba.cache
Loading model cost 1.020 seconds.
Prefix dict has been built succesfully.


In [7]:
def MakeWordsSet(words_file):
    words_set = set()  
    with open(words_file, 'r', encoding='utf-8') as f:  
        for line in f.readlines():  
            word = line.strip()  
            if len(word) > 0:  
                words_set.add(word)
    return words_set  

In [8]:
def words_dict(new_data_list, stopwords_set=set()):
    data_featrue_list = []  
    for t in range(1, len(new_data_list)):
        # 如果不是指定的结束语，并且单词长度大于1小于5，作为特征词
        if new_data_list[t] not in stopwords_set and 1 < len(new_data_list[t]) < 5:
            data_featrue_list.append(new_data_list[t])
    return data_featrue_list

In [9]:
# 生成stopwords_set
stopwords_file = './data/stopwords_cn.txt'
stopwords_set = MakeWordsSet(stopwords_file)


In [10]:
data_featrue_list = []
for i in range(len(new_data_list_cut)):
    data_featrue = words_dict(new_data_list_cut[i], stopwords_set)
    data_featrue_list.append(data_featrue)

In [11]:
# print(new_class_list[199:201])
# print(data_featrue_list[199:201])

In [12]:
cc = Counter(new_class_list)
cc.most_common()

[('企业所得税', 1000), ('增值税', 1000), ('个人所得税', 1000), ('综合', 1000)]

In [16]:
new_class_list[9]

'企业所得税'

In [13]:
data_class_list = list(zip(data_featrue_list, new_class_list))  # zip压缩合并，将数据与标签对应压缩
random.shuffle(data_class_list)  # 将data_class_list乱序
index = int(len(data_class_list) * 0.25) + 1  # 训练集和测试集切分的索引值
train_list = data_class_list[index:]  # 训练集
test_list = data_class_list[:index]  # 测试集
train_data_list, train_class_list = zip(*train_list)  # 训练集解压缩
test_data_list, test_class_list = zip(*test_list)  # 测试集解压缩

In [19]:
print(len(data_featrue_list))
print(len(new_class_list))

4000
4000


In [39]:
print([data_featrue_list[i], new_class_list[4]])

[['国税', '16', '汇算', '答疑', '购买', '预付卡', '相关', '内部', '外部', '凭证', '证明', '预付卡', '所有权', '发生', '转移', '按规定', '进行', '税前', '扣除', '政策', '是否', '全国', '引用', '是否', '全国性', '文件', '描述', '政策', '属于', '地方', '省市', '税务机关', '解答', '没有', '正式', '文件', '税务总局', '没有', '明确', '全国', '引用', '各地', '税务机关', '实际', '情况', '征管', '作出', '要求', '了解', '目前', '实务', '倾向', '天津', '国税', '答复', '方式', '操作', '企业', '税务机关', '没有', '统一', '明确', '建议', '税务机关', '要求', '为准'], '企业所得税']


In [74]:
train_data = []
train_data.append(["content", "label"])
for i in range(len(train_data_list)):
    k = ''.join(train_data_list[i])
    train_data.append([k, train_class_list[i]])

In [75]:
train_data[0:2]

[['content', 'label'],
 ['公司一个小型联欢会账务处理福利费费用认同职工文艺活动经费职工福利开支分录管理费用职工福利费银行存款', '企业所得税']]

In [67]:
import csv
with open('./data/train_taxdata_0102.csv', "w", newline='') as f:
    writer = csv.writer(f)
    for i in range(len(train_data)):
        writer.writerows(train_data[i])
    f.close()

In [69]:
train_text = []
with open("./data/train_taxdata_0102.csv",'r') as fr:
    reader = csv.reader(fr)
    for item in reader:
        train_text.append(item)
print(train_text[0:2])

[['c', 'o', 'n', 't', 'e', 'n', 't'], ['l', 'a', 'b', 'e', 'l']]


In [71]:
datas = [['name', 'age'],
         ['Bob', 14],
         ['Tom', 23],
        ['Jerry', '18']]

with open('example.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    for row in datas:
        writer.writerow(row)
    f.close()
    
train_text = []
with open("example.csv",'r') as fr:
    reader = csv.reader(fr)
    for item in reader:
        train_text.append(item)
print(train_text[0:2])

[['name', 'age'], ['Bob', '14']]


In [18]:
with open('./pickle/content_list_0102.pickle', 'wb') as file:
    pickle.dump(data_featrue_list, file)
with open('./pickle/label_list_0102.pickle', 'wb') as file:
    pickle.dump(new_class_list, file)
